In [1]:
import notebook_importer

In [2]:
import spdz
import random
import numpy as np
import zmq
import time

importing notebook from spdz.ipynb


In [3]:
# Alice is party 0
party = 0

# Connect to zmq
context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect ("tcp://localhost:5555")

#TODO: tmp solution remove ASAP
spdz.spdz_socket = socket
spdz.socket_party = party

In [4]:
# seed random numbers to make calculation
# deterministic (just a good practice)
random.seed(1)
np.random.seed(1)

# initialize weights randomly with mean 0
syn0 = (2*np.random.random((3,1)) - 1) / 1000
syn0_alice, syn0_bob = spdz.secure(syn0)

In [5]:
# Alice sends initial weights to Bob
print(spdz.swap_shares(syn0_bob, party, socket))

# Alice receives X and y data from Bob
X_alice = spdz.swap_shares(np.array("OK"), party, socket)
y_alice = spdz.swap_shares(np.array("OK"), party, socket)

OK


In [ ]:
# Multiplication Test

a, b = spdz.PrivateValue.secure(5)
t = spdz.swap_shares(b, party, socket)
res = a * t
other = spdz.swap_shares(res, party, socket)

In [6]:
sigmoid = spdz.SigmoidInterpolated10(party, socket)
result = sigmoid.evaluate(X_alice)
result_bob = spdz.swap_shares(result, party, socket)

network = spdz.TwoLayerNetwork(sigmoid, party, socket)

start_time = time.time()
network.train(X_alice, y_alice, syn0_alice)

print("Elapsed Training Time:\n", time.time() - start_time)
network.print_weights()

preds = network.predict(X_alice)
preds_bob = spdz.swap_shares(preds, party, socket)

TypeError: can only concatenate list (not "int") to list